<a href="https://colab.research.google.com/github/miki727/ABCNY_Project/blob/main/Machine%20learning/Shuing%20Ma/ml_model_binary_noscore_piml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
!pip install jellyfish
!pip install fuzzywuzzy
!pip install PiML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import time
import re
import pandas as pd
import numpy as np
import jellyfish as jf
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [4]:
df = pd.read_csv('ml_dataset.csv')
df

,Original Name,Test Case Name,Test Type,Label
0,JAM'YAH TA'AWUN AL-ISLAMIA,JAM'YAH TA'AWUN AL-ISLAMIA,Fuzzy Match,Match
1,"GLOBOVISION TELE CA, CORP.","GLOBOVISION TELE CA, CORP.",Fuzzy Match,Match
2,KOREAN PEOPLE'S ARMY,KOREAN PEOPLE'S ARMY,Fuzzy Match,Match
3,RANCHO LA HERRADURA,RANCHO LA HERRADURA,Fuzzy Match,Match
4,"TECHNO ENERGY, S.A.","TECHNO ENERGY, S.A.",Fuzzy Match,Match
...,...,...,...,...
18503,GLOBAL AGE LIMITED,"SHAABAN, Bouthaina",Non-Match,Not Match
18504,OBSHCHESTVO S OGRANICHENNOI OTVETSTVENNOSTYU M...,TAMIN KALAYE SABZ ARAS COMPANY,Non-Match,Not Match
18505,"AHMED, Qassim Abdullah Ali",KOREA HAEGUMGANG TRADING CORPORATION,Non-Match,Not Match
18506,JINGHO TECHNOLOGY CO. LIMITED,COMITE' DE BIENFAISANCE ET DE SECOURS AUX PALE...,Non-Match,Not Match


# Feature engineering

In [5]:
def matching_numbers(original_name, test_case_name):

    original_numbers = set(re.findall(r'[0-9]+', original_name))
    test_case_numbers = set(re.findall(r'[0-9]+', test_case_name))    
    union = original_numbers.union(test_case_numbers)
    intersection = original_numbers.intersection(test_case_numbers)

    if len(original_numbers)==0 and len(test_case_numbers) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))

In [6]:
def engineer_features(df):

    df['Test Case Name'] = df['Test Case Name'].str.lower()
    df['Original Name'] = df['Original Name'].str.lower()

    df['levenshtein_distance'] = df.apply(
    lambda x: jf.levenshtein_distance(x['Original Name'], 
                                      x['Test Case Name']), axis=1)

    df['damerau_levenshtein_distance'] = df.apply(
    lambda x: jf.damerau_levenshtein_distance(x['Original Name'], 
                                              x['Test Case Name']), axis=1)

    df['hamming_distance'] = df.apply(
    lambda x: jf.hamming_distance(x['Original Name'], 
                                  x['Test Case Name']), axis=1)

    df['jaro_similarity'] = df.apply(
    lambda x: jf.jaro_similarity(x['Original Name'], 
                                  x['Test Case Name']), axis=1)

    df['jaro_winkler_similarity'] = df.apply(
    lambda x: jf.jaro_winkler_similarity(x['Original Name'], 
                                         x['Test Case Name']), axis=1)

    df['match_rating_comparison'] = df.apply(
    lambda x: jf.match_rating_comparison(x['Original Name'], 
                                         x['Test Case Name']), axis=1).fillna(0).astype(int)

    df['ratio'] = df.apply(
    lambda x: fuzz.ratio(x['Original Name'], 
                         x['Test Case Name']), axis=1)

    df['partial_ratio'] = df.apply(
    lambda x: fuzz.partial_ratio(x['Original Name'], 
                                 x['Test Case Name']), axis=1)

    df['token_sort_ratio'] = df.apply(
    lambda x: fuzz.token_sort_ratio(x['Original Name'], 
                                    x['Test Case Name']), axis=1)

    df['token_set_ratio'] = df.apply(
    lambda x: fuzz.token_set_ratio(x['Original Name'], 
                                   x['Test Case Name']), axis=1)

    df['w_ratio'] = df.apply(
    lambda x: fuzz.WRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)

    df['uq_ratio'] = df.apply(
    lambda x: fuzz.UQRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)

    df['q_ratio'] = df.apply(
    lambda x: fuzz.QRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)    

    df['matching_numbers'] = df.apply(
    lambda x: matching_numbers(x['Original Name'], 
                               x['Test Case Name']), axis=1)

    df['matching_numbers_log'] = (df['matching_numbers']+1).apply(np.log)

    df['log_fuzz_score'] = (df['ratio'] + df['partial_ratio'] + 
                            df['token_sort_ratio'] + df['token_set_ratio']).apply(np.log)

    df['log_fuzz_score_numbers'] = df['log_fuzz_score'] + (df['matching_numbers']).apply(np.log)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value=0, inplace=True)

    return df

In [7]:
df = engineer_features(df)
df

,Original Name,Test Case Name,Test Type,Label,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,match_rating_comparison,...,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,Match,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,Match,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,Match,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,Match,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,Match,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,Not Match,16,16,18,0.449735,0.449735,0,...,30,23,23,28,28,28,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,Not Match,44,44,51,0.510733,0.510733,0,...,27,25,25,28,20,20,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,Not Match,28,28,35,0.553419,0.553419,0,...,23,10,10,23,23,23,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,Not Match,43,43,54,0.546201,0.546201,0,...,24,30,31,33,17,17,1.0,0.693147,4.624973,4.624973


In [8]:
df['Label'] = df['Label'].map({'Match' : 1, 'Not Match' : 0})
df

,Original Name,Test Case Name,Test Type,Label,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,match_rating_comparison,...,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,1,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,1,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,1,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,1,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,1,0,0,0,1.000000,1.000000,1,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,0,16,16,18,0.449735,0.449735,0,...,30,23,23,28,28,28,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,0,44,44,51,0.510733,0.510733,0,...,27,25,25,28,20,20,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,0,28,28,35,0.553419,0.553419,0,...,23,10,10,23,23,23,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,0,43,43,54,0.546201,0.546201,0,...,24,30,31,33,17,17,1.0,0.693147,4.624973,4.624973


# Select the model

In [9]:
from piml import Experiment
exp = Experiment()

In [10]:
exp.data_loader(data=df)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,Original Name,Test Case Name,Test Type,Label,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,match_rating_comparison,ratio,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,1.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,1.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,1.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,1.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,1.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,0.0,16.0,16.0,18.0,0.449735,0.449735,0.0,28.0,30.0,23.0,23.0,28.0,28.0,28.0,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,0.0,44.0,44.0,51.0,0.510733,0.510733,0.0,20.0,27.0,25.0,25.0,28.0,20.0,20.0,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,0.0,28.0,28.0,35.0,0.553419,0.553419,0.0,23.0,23.0,10.0,10.0,23.0,23.0,23.0,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,0.0,43.0,43.0,54.0,0.546201,0.546201,0.0,17.0,24.0,30.0,31.0,33.0,17.0,17.0,1.0,0.693147,4.624973,4.624973


In [11]:
exp.data_summary(feature_exclude=["Original Name", "Test Case Name", "Test Type"])

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value='Data Shape:(18508, 21)')

In [12]:
exp.data_prepare(target='Label', task_type='Classification', test_ratio=0.3, random_state=0)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,0,1
0,Excluded columns,"[Original Name, Test Case Name, Test Type]"
1,Target variable,Label
2,Sample weight,None
3,Task type,Classification
4,Split method,Random
5,Test ratio,0.3
6,Random state,0


In [13]:
# exp.feature_select(threshold=0.98)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [14]:
exp.eda()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

In [15]:
## ReLU-DNN: ReLU Deep Neural Networks with L1-Regularization, e.g. L1_reg = 0.0008
from piml.models import ReluDNNClassifier

clf1 = ReluDNNClassifier(hidden_layer_sizes=(40, 40), l1_reg=0.0008,
                        batch_size=500, learning_rate=0.001)
exp.model_train(model=clf1, name='ReLU-DNN')
exp.model_diagnose(model="ReLU-DNN", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register ReLU-DNN Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,0.9893,0.9974,0.9863,0.9923,0.9893
Test,0.9933,0.9982,0.9920,0.9945,0.9933
Gap,0.0041,0.0007,0.0057,0.0023,0.0040


In [16]:
## GAMI-Net: GAM-Neural Networks with Structured Interactions
from piml.models import GAMINetClassifier

clf2 = GAMINetClassifier(interact_num=10, loss_threshold=0.01,  
                        subnet_size_main_effect=[20], 
                        subnet_size_interaction=[20,20])
exp.model_train(model=clf2, name='GAMI-Net')
exp.model_diagnose(model="GAMI-Net", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register GAMI-Net Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,0.9920,0.9987,0.9882,0.9960,0.9921
Test,0.9951,0.9995,0.9938,0.9963,0.9951
Gap,0.0031,0.0008,0.0056,0.0004,0.0030


In [17]:
## EBM: Explainable Boosting Machine from MS Research
from piml.models import ExplainableBoostingClassifier

clf3 = ExplainableBoostingClassifier(interactions=10, random_state=0)
exp.model_train(model=clf3, name='EBM')
exp.model_diagnose(model="EBM", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register EBM Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,0.9945,0.9995,0.9920,0.9971,0.9945
Test,0.9950,0.9993,0.9934,0.9963,0.9949
Gap,0.0004,-0.0002,0.0014,-0.0007,0.0004


In [18]:
exp.model_compare(models=['ReLU-DNN', 'GAMI-Net', 'EBM'], show='accuracy')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

In [19]:
exp.model_explain()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>